## Import Statements
### The libraries used are `pandas`, `pathlib`, `imblearn`, and `sklearn`. 

In [ ]:
import pandas as pd
from pathlib import Path
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

## Import the data and create a dataframe
1. Use `pandas` and `pathlib` to read the `BankChurnersPrimary.csv` CSV from the `Resources` folder.
2. Sample the dataset to review the data.

In [ ]:
starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/BankChurnersPrimary.csv"))
starter_df.sample(5)

1519 Unknowns in Education

749 Unknowns in Marital_Status

1112 Unknowns in Income_Category



In [ ]:
starter_df = starter_df.drop(columns=['Education_Level', 'Marital_Status', 'Income_Category'])

## Drop any Unknown or NaN values

In [ ]:
#starter_df = starter_df[starter_df != 'Unknown'].dropna()

## Remove any columns that won't be used.
Naive Bayes columns can't be used (it's calculated from features and target).

CLIENTNUM is irrelevant.

In [ ]:
starter_df.drop(columns=[
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
'CLIENTNUM'
], 
inplace= True)

## Evaluate the Education column for use as a feature
The Education values are ranked from uneducated to doctorate, and is a gradient rather than ambiguous classifications. 

The ranks will be converted to numericals and specified as datatype `int`.
1. Uneducated = 0
2. Highschool = 1
3. College = 2
4. Graduate = 3
5. Post-Grad = 4
6. Doctorate = 5

In [ ]:
'''
starter_df = starter_df.replace({'Education_Level' : {
'Uneducated': 0, 
'High School': 1, 
'College' : 2, 
'Graduate' :  3, 
'Post-Graduate' : 4, 
'Doctorate' : 5
}})
starter_df.Education_Level = starter_df.Education_Level.astype(int)
'''

## Evaluate the Card Type column for use as a feature
The Card Type values are ranked from blue to platinum, and is a gradient rather than ambiguous classifications. The ranks will be converted to numericals and specified as datatype `int`.
1. Blue = 0
2. Silver = 1
3. Gold = 2
4. Platinum = 3


In [ ]:
starter_df = starter_df.replace({'Card_Category' : {
'Blue': 0, 
'Silver': 1, 
'Gold' : 2, 
'Platinum' : 3
}})
starter_df.Card_Category = starter_df.Card_Category.astype(int)

## Evaluate the Gender column for use as a feature
Gender is a binary classification, so we don't need to OneHotEncode.
The values will be converted to numericals and specified as datatype `int`.
1. Male = 0
2. Female = 1 


In [ ]:
starter_df = starter_df.replace({'Gender' : {
'M': 0, 
'F': 1
}})
starter_df.Gender = starter_df.Gender.astype(int)

## Evaluate the Attrition_Flag column for use as the target
Attrition_Flag is a binary classification, so we don't need to OneHotEncode.
The values will be converted to numericals and specified as datatype `int`.
1. Attrited Customer = 0
2. Existing Customer = 1 

In [ ]:
starter_df = starter_df.replace({'Attrition_Flag' : {
'Attrited Customer': 0, 
'Existing Customer': 1
}})
starter_df.Attrition_Flag = starter_df.Attrition_Flag.astype(int)

## Perform feature creation on the Dependents and Marital_Status features to create the Marital_Dependent_Ratio feature
The values will be converted to numericals and specified as datatype `int`.
Divorced and Single will be combined, as they are both single income sources.
1. Divorced = 1
2. Single = 1 
3. Married = 2

The formula for Marital_Dependent_Ratio is as follows:

( Marital_Status / (Dependents + 1) / 2 )

The original Marital_Status and Dependents columns will be dropped in place of the Marital_Dependent_Ratio column.

In [ ]:
'''
starter_df = starter_df.replace({'Marital_Status' : {
'Divorced': 1, 
'Single': 1, 
'Married' : 2
}})
starter_df.Marital_Status = starter_df.Marital_Status.astype(int)
starter_df['Marital_Dependent_Ratio'] = ((starter_df['Marital_Status'] / (starter_df['Dependent_count'] + 1)) / 2).round(2)
starter_df.drop(columns=['Marital_Status', 'Dependent_count'], inplace=True)
'''





## Perform feature creation on the Avg_Open_To_Buy and Credit_Limit features to create the Avg_Trans_Value feature. Its a simple ratio.
The original Avg_Open_To_Buy and Credit_Limit columns will be dropped in place of the Credit_Usage column.

In [ ]:
starter_df['Credit_Usage'] = (starter_df['Avg_Open_To_Buy'] / starter_df['Credit_Limit']).round(2)
starter_df.drop(columns=['Avg_Open_To_Buy', 'Credit_Limit'], inplace=True)

## Perform feature creation on the Total_Trans_Amt and Total_Trans_Ct features to create the Avg_Trans_Value feature. Its a simple ratio.
The original Total_Trans_Amt and Total_Trans_Ct columns will be dropped in place of the Avg_Trans_Value column.

In [ ]:
starter_df['Avg_Trans_Value'] = (starter_df['Total_Trans_Amt'] / starter_df['Total_Trans_Ct']).round(2)
starter_df.drop(columns=['Total_Trans_Amt', 'Total_Trans_Ct'], inplace=True)

## Perform feature creation on the Tenure(month) and Age features to create the Tenure_By_Age feature. Its a simple ratio.

The original Tenure(month) and Age columns will be dropped in place of the Tenure_By_Age column.

In [ ]:
#Do feature creation to make a ratio between age and tenure
starter_df['Tenure_By_Age'] = (starter_df['Months_on_book'] / starter_df['Customer_Age']).round(2)
starter_df.drop(columns=['Months_on_book', 'Customer_Age'], inplace=True)

## Evaluate the Income column for use as the target
 The Income values are ranked in blocks, and is a gradient rather than ambiguous classifications. The ranks will be converted to numericals and specified as datatype `int`.

In [ ]:
'''
starter_df = starter_df.replace({'Income_Category' : {
'Less than $40K': 0, 
'$40K - $60K': 1, 
'$80K - $120K' : 2, 
'$60K - $80K' : 3, 
'$120K +' : 4
}})
starter_df.Income_Category = starter_df.Income_Category.astype(int)
'''

## Review the new features

In [ ]:
starter_df.dtypes

In [ ]:
starter_df.sample(5)

## Define our features(X) and our target(y)

In [ ]:
y = starter_df['Attrition_Flag']
X = starter_df.drop(columns=['Attrition_Flag'])

## Split features and target in to training and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Using StandardScaler, scale all numerical/float values that don't represent ambiguous categories
### StandardScaler will be used within ColumnTransformer. This allows for the scaling of specified columns within our X_train and X_test set.
It's critical that the StandardScaler is only fit to our X_train. Both X_train and X_test will be transformed, but the StandardScaler will only be fit to X_train.This prevents the model from 'cheating'.

In [ ]:
scaler = StandardScaler()

col_tran= ColumnTransformer([
('Total_Revolving_Bal_Scaled', scaler, ['Total_Revolving_Bal']),
#('Education_Scaled', scaler, ['Education_Level']),
#('Income_Scaled', scaler, ['Income_Category']),
('Card_Type_Scaled', scaler, ['Card_Category']),
('Months_Inactive_12_mon_scaled', scaler, ['Months_Inactive_12_mon']),
('Credit_Usage_Scaled', scaler, ['Credit_Usage']),
('Avg_Trans_Value_Scaled', scaler, ['Avg_Trans_Value']),
('Tenure_By_Age_Scaled', scaler, ['Tenure_By_Age']),
('Avg_Util_Ratio_Scaled', scaler, ['Avg_Utilization_Ratio']),
('Total_Relationship_Count_Scaled', scaler, ['Total_Relationship_Count']),
('Contacts_Count_12_mon_scaled', scaler, ['Contacts_Count_12_mon']),
('Total_Amt_Chng_Q4_Q1_scaled', scaler, ['Total_Amt_Chng_Q4_Q1']),
('Total_Ct_Chng_Q4_Q1_scaled', scaler, ['Total_Ct_Chng_Q4_Q1']),
#('Marital_Dependent_Ratio_Scaled', scaler, ['Marital_Dependent_Ratio']),
('Dependent_count', scaler, ['Dependent_count'])
])


X_train = col_tran.fit_transform(X_train)
X_test = col_tran.transform(X_test)

## Use SMOTE to add synthetic data and balance our target/feature value count
This will only be done on the training data.

In [ ]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
parameters = {
#'n_estimators' : [100, 300],
#'random_state' : [1, 2],
'subsample' : [1, 0.9],
'min_samples_split' : [2, 3],
#'max_depth' : [3, 2],
'min_impurity_decrease' : [0, 1],
#'min_samples_leaf' : [1, 2],
'min_weight_fraction_leaf' : [0, 0.1],
'max_leaf_nodes' : [None, 2],
#'learning_rate' : [0.1, 0.0],
'criterion' : ('friedman_mse', 'squared_error'),
'init' : ('zero', None),
'loss' : ('log_loss', 'deviance', 'exponential')
}



gbc = GradientBoostingClassifier(n_estimators=500, random_state=2)
clf = GridSearchCV(gbc, parameters)
clf.fit(X_train, y_train)

In [ ]:
sorted(clf.cv_results_.keys())

## Create an instance of the GradientBoostingClassifier model
Hyperparameter tuning was mostly unhelpful; only two hyperparameters have been changed from default values.
1. n_estimators = 500
2. random_state= 2

In [ ]:
gbc_model = GradientBoostingClassifier(
n_estimators=500,                 #default = 100    range = 1-inf
random_state= 2,                  #default = None   range = 1-inf
subsample= 1,                     #default = 1      range = 0.-1
min_samples_split = 2,            #default = 2      range = 2-inf
max_depth=3,                      #default = 3      range = 1-inf
min_impurity_decrease=0,          #default = 0      range = 0 - inf
min_samples_leaf = 1,             #default = 1      range = 1 - inf
min_weight_fraction_leaf = 0,     #default = 0      range =0 - 0.5
max_leaf_nodes = None,            #default = None   range = 2-inf
learning_rate = 0.1               #default=0.1      range 0.0-inf
)

## Fit the model on the training data

In [ ]:
gbc_model.fit(X_train, y_train)

## Make predictions on the test data

In [ ]:
gbc_test_predictions = gbc_model.predict(X_test)

## Evaluate the model's performance

In [ ]:
accuracy = accuracy_score(y_test, gbc_test_predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))

## Generate a confusion matrix

In [ ]:
gbc_test_matrix = confusion_matrix(y_test, gbc_test_predictions)
print(gbc_test_matrix)

## Generate a classification report

In [ ]:
gbc_testing_report = classification_report(y_test, gbc_test_predictions)
print(gbc_testing_report)

## Generate an imbalanced classification report

In [ ]:
imb__gbc_testing_report = classification_report_imbalanced(y_test, gbc_test_predictions)
print(imb__gbc_testing_report)